# Experiment
Fine tuning NVIDIA STT Fast Conformer Transducer

This time with a hopefully better audio processing.

In [35]:
!conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pytorch
    - pytorch-cuda=11.8
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cuda-cudart-11.8.89        |                0         197 KB  nvidia
    cuda-cupti-11.8.87         |                0        25.3 MB  nvidia
    cuda-libraries-11.8.0      |                0           1 KB  nvidia
    cuda-nvrtc-11.8.89         |                0        19.1 MB  nvidia
    cuda-nvtx-11.8.86          |                0          57 KB  nvidia
    cuda-runtime-11.8.0        |                0           1 KB  nvidia
    libcublas-11.11.3.6        |          

In [42]:
!conda install cudatoolkit -y

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - cudatoolkit


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cudatoolkit-11.8.0         |       h6a678d5_0       630.7 MB
    ------------------------------------------------------------
                                           Total:       630.7 MB

The following NEW packages will be INSTALLED:

  cudatoolkit        pkgs/main/linux-64::cudatoolkit-11.8.0-h6a678d5_0 

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2023.5.7~ --> pkgs/main::ca-certificates-2023.01.10-h06a4308_0 
  certifi            conda-forge/noarch::certifi-2023.5.7-~ --> pkgs/main/linux-64::certifi-2023.5.7-py310h06a4308_0 
  conda              conda-forge::conda-23.3.1-py310hff520~ --> pkgs/main::conda-23.3.1-py310h06a4308_0 




In [3]:
%pip install Cython
%pip install nemo_toolkit['all']

  Using cached Cython-0.29.34-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
Note: you may need to restart the kernel to use updated packages.
  Using cached nemo_toolkit-1.18.1-py3-none-any.whl (2.3 MB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
  Using cached wget-3.2-py3-none-any.whl
  Using cached scikit_learn-1.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Using cached onnx-1.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.6 MB)
  Using cached setuptools-65.5.1-py3-none-any.whl (1.2 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached numba-0.57.0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.6 MB)
  Using cached wrapt-1.15.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (78 kB)
  Using cached gdown-4.7.1-py3-none-an

In [4]:
%pip install jiwer librosa nemo_text_processing

Note: you may need to restart the kernel to use updated packages.


In [9]:
!conda install -y -c conda-forge libstdcxx-ng=12

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - libstdcxx-ng=12


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.5.7   |       hbcca054_0         145 KB  conda-forge
    certifi-2023.5.7           |     pyhd8ed1ab_0         149 KB  conda-forge
    conda-23.3.1               |  py310hff52083_0         941 KB  conda-forge
    libstdcxx-ng-12.2.0        |      h46fd767_19         4.3 MB  conda-forge
    python_abi-3.10            |          2_cp310           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.5 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/linux-64::python_abi-3.10-2_cp310 

The following packages will be UPDATED:

  ca-certificates    pkgs/main::ca-certificate

In [56]:
%pip install pyloudnorm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 15.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492025 sha256=94938acb28f18591529e63bf2831a21d3194814df7273373ebe670dc204e2ae5
  Stored in directory: /root/.cache/pip/wheels/5e/a9/47/f118e66afd12240e4662752cc22cefae5d97275623aa8ef57d
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import copy
import json
import math
import librosa
import numpy as np
import pandas as pd

# Model Training
import nemo.collections.asr as nemo_asr
# NeMo's Experiment manager to handle checkpoint saving and logging
from nemo.utils import exp_manager
from omegaconf import OmegaConf, open_dict
from pytorch_lightning import Trainer, seed_everything

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio

# Model Evaluation
from jiwer import wer

# Reproducibility
seed_everything(42, workers=True)

[NeMo W 2023-05-21 09:26:04 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-05-21 09:26:05 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.
Global seed set to 42


42

### Load Dataset

In [2]:
# Define paths to data sources
DATASET_TRAIN = 'data/train/Train.csv'
FILE_TEST = 'data/test/SampleSubmission_Advanced.csv'
TRAIN_DATA_DIR = 'data/train/'
TEST_DATA_DIR = 'data/test/'

In [3]:
# Load dataset
dataset = pd.read_csv(DATASET_TRAIN)
# Shuffle the records
dataset = dataset.sample(frac=1, random_state=1).reset_index(drop=True)
# Load file for storing model predictions
test_set = pd.read_csv(FILE_TEST)

In [4]:
dataset["annotation"] = dataset["annotation"].str.lower() # normalize to lower case

In [5]:
# Define train test split
TRAIN_TEST_SPLIT = 0.8

# Split the dataset
train_proportion = int(TRAIN_TEST_SPLIT * len(dataset))
train_set = dataset[:train_proportion]
dev_set = dataset[train_proportion:]

In [6]:
# Build manifest files for training with NVIDIA STT models
def build_manifest(ds, manifest_path, is_train_set):
    with open(manifest_path, 'w') as mp:
        for idx, record in ds.iterrows():
            # Get audio file
            audio_path = record['path']

            if is_train_set:
                # Get duration
                duration = librosa.core.get_duration(filename=f'{TRAIN_DATA_DIR}{audio_path}')
                # Get transcript
                transcript = record['annotation'].lower() # normalize to lower case
                metadata = {
                    "audio_filepath": f'{TRAIN_DATA_DIR}{audio_path}',
                    "duration": duration,
                    "text": transcript
                }
            else:
                # Get duration 
                duration = librosa.core.get_duration(filename=f'{TEST_DATA_DIR}audio/{audio_path}')
                metadata = {
                    "audio_filepath": f'{TEST_DATA_DIR}audio/{audio_path}',
                    "duration": duration
                } 

            json.dump(metadata, mp)
            mp.write('\n')

# Create manifests file if they haven't been created
train_manifest_file = os.path.join(TRAIN_DATA_DIR, 'train_manifest.json')
if not os.path.isfile(train_manifest_file):
    build_manifest(train_set, train_manifest_file, is_train_set=True)

val_manifest_file = os.path.join(TRAIN_DATA_DIR, 'val_manifest.json')
if not os.path.isfile(val_manifest_file):
    build_manifest(dev_set, val_manifest_file, is_train_set=True)

test_manifest_file = os.path.join(TEST_DATA_DIR, 'test_manifest.json')
if not os.path.isfile(test_manifest_file):
    build_manifest(test_set, test_manifest_file, is_train_set=False)

print('***Manifests created***')

***Manifests created***


### Prepare Model

#### Tokenizer

In [7]:
# Create directory for nemo scripts
if not os.path.exists("scripts/"):
  os.makedirs("scripts")

In [8]:
# Leverage modern encoder architecture like Conformer-T through a tokenizer based model
if not os.path.exists("scripts/process_asr_text_tokenizer.py"):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tokenizers/process_asr_text_tokenizer.py

In [9]:
VOCAB_SIZE = 32000
TOKENIZER_TYPE = "spe"

if not os.path.exists("tokenizers"):
  os.makedirs("tokenizers")

!python scripts/process_asr_text_tokenizer.py \
   --manifest=$train_manifest_file \
   --data_root="tokenizers" \
   --tokenizer=$TOKENIZER_TYPE \
   --log \
   --vocab_size=$VOCAB_SIZE

[NeMo W 2023-05-21 09:26:10 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
INFO:root:Finished extracting manifest : data/train/train_manifest.json
INFO:root:Finished extracting all manifests ! Number of sentences : 3000
[NeMo I 2023-05-21 09:26:10 sentencepiece_tokenizer:315] Processing tokenizers/text_corpus/document.txt and store at tokenizers/tokenizer_spe_bpe_v32000
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tokenizers/text_corpus/document.txt --model_prefix=tokenizers/tokenizer_spe_bpe_v32000/tokenizer --vocab_size=32000 --shuffle_input_sentence=true --hard_vocab_limit=false --model_type=bpe --character_coverage=1.0 --bos_id=-1 --eos_id=-1 --normalization_rule_name=nmt_nfkc_cf
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: tokenizers/text_corpus/document.txt
  input_format: 
  model_prefix: tokenizers/tokenizer_spe_bpe_v32000/tokenizer
  model_type: BPE
  vocab_size: 32000
  se

In [10]:
# Tokenizer path
TOKENIZER = os.path.join("tokenizers", f"tokenizer_spe_bpe_v32000")
TOKENIZER_TYPE_CFG = "spe"

#### Pretrained Model

In [11]:
checkpoint = None
if checkpoint is None:
    asr_model = nemo_asr.models.ASRModel.from_pretrained(model_name='stt_en_fastconformer_transducer_large')
else:
    asr_model = nemo_asr.models.ASRModel.restore_from(checkpoint)

[NeMo I 2023-05-21 09:26:27 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.18.1/stt_en_fastconformer_transducer_large/74b4e49a2b22465da737dd5345767e61/stt_en_fastconformer_transducer_large.nemo.
[NeMo I 2023-05-21 09:26:27 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.18.1/stt_en_fastconformer_transducer_large/74b4e49a2b22465da737dd5345767e61/stt_en_fastconformer_transducer_large.nemo
[NeMo I 2023-05-21 09:26:27 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-05-21 09:26:27 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-05-21 09:26:28 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 1
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: fully_randomized
    bucketing_batch_size: null
    
[NeMo W 2023-05-21 09:26:28 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    num_workers: 8
    pin_m

[NeMo I 2023-05-21 09:26:28 features:287] PADDING: 0


[NeMo W 2023-05-21 09:26:29 nemo_logging:349] /opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    


[NeMo I 2023-05-21 09:26:30 rnnt_models:206] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2023-05-21 09:26:30 save_restore_connector:249] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.18.1/stt_en_fastconformer_transducer_large/74b4e49a2b22465da737dd5345767e61/stt_en_fastconformer_transducer_large.nemo.


#### Freeze Encoder
Since the TIL dataset is quite small, it is better to freeze the encoder.

In [12]:
def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

In [13]:
freeze_encoder = True
if freeze_encoder:
    asr_model.encoder.freeze()
    asr_model.encoder.apply(enable_bn_se)
    print("Model encoder has been frozen")
else:
    asr_model.encoder.unfreeze()
    print("Model encoder has been un-frozen")

Model encoder has been frozen


### Configure Model

In [14]:
cfg = copy.deepcopy(asr_model.cfg)

In [15]:
cfg.tokenizer.dir = TOKENIZER
cfg.tokenizer.type = TOKENIZER_TYPE_CFG

# Set tokenizer config
asr_model.cfg.tokenizer = cfg.tokenizer

In [16]:
print(OmegaConf.to_yaml(cfg.tokenizer))

dir: tokenizers/tokenizer_spe_bpe_v32000
type: spe
model_path: nemo:e7bbc43ca90943c989b60aece82c16c8_tokenizer.model
vocab_path: nemo:b7966c4186cd467b8e612a7d9384f273_vocab.txt
spe_tokenizer_vocab: nemo:0e7783691b714849aa760378530c1f39_tokenizer.vocab



In [17]:
# Setup train, validation, test configs
with open_dict(cfg):    
  # Train dataset
  cfg.train_ds.manifest_filepath = train_manifest_file
  cfg.train_ds.normalize_transcripts = False
  cfg.train_ds.batch_size = 32
  cfg.train_ds.num_workers = 8
  cfg.train_ds.pin_memory = True
  cfg.train_ds.trim_silence = True

  # Validation dataset
  cfg.validation_ds.manifest_filepath = val_manifest_file
  cfg.validation_ds.normalize_transcripts = False
  cfg.validation_ds.batch_size = 8
  cfg.validation_ds.num_workers = 8
  cfg.validation_ds.pin_memory = True
  cfg.validation_ds.trim_silence = True

  # Test dataset
  cfg.test_ds.manifest_filepath = test_manifest_file
  cfg.test_ds.normalize_transcripts = False
  cfg.test_ds.batch_size = 8
  cfg.test_ds.num_workers = 8
  cfg.test_ds.pin_memory = True
  cfg.test_ds.trim_silence = True

In [18]:
# Setup data loaders with new configs
asr_model.setup_training_data(cfg.train_ds)
asr_model.setup_validation_data(cfg.validation_ds)
asr_model.setup_test_data(cfg.test_ds)

[NeMo I 2023-05-21 09:26:43 collections:193] Dataset loaded with 3000 files totalling 3.31 hours
[NeMo I 2023-05-21 09:26:43 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-05-21 09:26:43 collections:193] Dataset loaded with 750 files totalling 0.82 hours
[NeMo I 2023-05-21 09:26:43 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-05-21 09:26:43 collections:193] Dataset loaded with 12000 files totalling 13.26 hours
[NeMo I 2023-05-21 09:26:43 collections:194] 0 files were filtered totalling 0.00 hours


In [19]:
with open_dict(asr_model.cfg.optim):
  asr_model.cfg.optim.lr = 5e-3 # from model config
  asr_model.cfg.optim.betas = [0.9, 0.98]  # from model config
  asr_model.cfg.optim.weight_decay = 1e-3  # Original weight decay
  asr_model.cfg.optim.sched.warmup_steps = None  # Remove default number of steps of warmup
  asr_model.cfg.optim.sched.warmup_ratio = 0.05  # 5 % warmup
  asr_model.cfg.optim.sched.min_lr = 5e-4  # from model config

In [20]:
# Original optimizer + scheduler
print(OmegaConf.to_yaml(asr_model.cfg.optim))

name: adamw
lr: 0.005
betas:
- 0.9
- 0.98
weight_decay: 0.001
sched:
  name: CosineAnnealing
  warmup_steps: null
  warmup_ratio: 0.05
  min_lr: 0.0005



In [21]:
asr_model.cfg.joint.fuse_loss_wer = True
asr_model.cfg.joint.fused_batch_size = 16

In [22]:
asr_model.cfg.compute_eval_loss = True

#### Configure Data Aug

In [23]:
augment_audio = False
if augment_audio:
    with open_dict(asr_model.cfg.spec_augment):
      asr_model.cfg.spec_augment.freq_masks = 2
      asr_model.cfg.spec_augment.freq_width = 25
      asr_model.cfg.spec_augment.time_masks = 2
      asr_model.cfg.spec_augment.time_width = 0.05
else:
    with open_dict(asr_model.cfg.spec_augment):
      asr_model.cfg.spec_augment.freq_masks = 0
      asr_model.cfg.spec_augment.time_masks = 0

In [24]:
print(OmegaConf.to_yaml(asr_model.cfg.spec_augment))

_target_: nemo.collections.asr.modules.SpectrogramAugmentation
freq_masks: 0
time_masks: 0
freq_width: 27
time_width: 0.05



### Training

In [34]:
if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'cpu'

EPOCHS = 10 # NOTE: Increase after ensuring that the code is working

# Initialize a Trainer for the Transducer model
trainer = Trainer(devices=1,
                  precision=16, # mixed precision
                  accelerator=accelerator,
                  max_epochs=EPOCHS,
                  enable_checkpointing=False,
                  logger=False,
                  log_every_n_steps=5,
                  check_val_every_n_epoch=1,
                  deterministic=True)

# Setup model with the trainer
asr_model.set_trainer(trainer)

# Update model's internal configuration
asr_model.cfg = asr_model._cfg

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [28]:
!wandb login 23c74d0b24c250fc7fe6df6c59e5ee5bca3340cd

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [32]:
%env WANDB_PROJECT=TIL2023-ASR
%env WANDB_NOTEBOOK_NAME=nvidia-stt-ft-02-spe

env: WANDB_PROJECT=TIL2023-ASR
env: WANDB_NOTEBOOK_NAME=nvidia-stt-ft-02-spe


In [35]:
# Environment variable generally used for multi-node multi-gpu training.
# In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
os.environ.pop('NEMO_EXPM_VERSION', None)

exp_config = exp_manager.ExpManagerConfig(
    exp_dir=f'experiments/',
    name=f"nvidia-stt-ft-02-spe",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
    create_wandb_logger=True,
    wandb_logger_kwargs={
        "project" : "TIL2023-ASR",
        "name": "nvidia-stt-ft-02-spe"
    }
)

exp_config = OmegaConf.structured(exp_config)

logdir = exp_manager.exp_manager(trainer, exp_config)

[NeMo I 2023-05-21 09:33:30 exp_manager:374] Experiments will be logged at experiments/nvidia-stt-ft-02-spe/2023-05-21_09-33-30
[NeMo I 2023-05-21 09:33:30 exp_manager:797] TensorboardLogger has been set up


[NeMo W 2023-05-21 09:33:30 nemo_logging:349] /opt/conda/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
      rank_zero_warn(
    


[NeMo I 2023-05-21 09:33:30 exp_manager:812] WandBLogger has been set up


In [36]:
%%time
trainer.fit(asr_model)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2023-05-21 09:33:34 modelPT:722] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.005
        maximize: False
        weight_decay: 0.001
    )
[NeMo I 2023-05-21 09:33:34 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7fa22fae9c30>" 
    will be used during training (effective maximum steps = 940) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.05
    min_lr: 0.0005
    max_steps: 940
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoder                  | 108 M 
2 | decoder           | RNNTDecoder                       | 3.9 M 
3 | joint             | RNNTJoint                         | 1.4 M 
4 | loss              | RNNTLoss                          | 0     
5 | spec_augmentation | SpectrogramAugmentation           | 0     
6 | wer               | RNNTBPEWER                        | 0     
------------------------------------------------------------------------
5.4 M     Trainable params
108 M     Non-trainable params
114 M     Total params
456.382   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[NeMo I 2023-05-21 09:33:45 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-21 09:33:45 rnnt_wer_bpe:395] reference :he received nine birthday cards in the mail
[NeMo I 2023-05-21 09:33:45 rnnt_wer_bpe:396] predicted :she received nine birthday cards in the mail
[NeMo I 2023-05-21 09:33:46 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-21 09:33:46 rnnt_wer_bpe:395] reference :she scored nine for the quiz
[NeMo I 2023-05-21 09:33:46 rnnt_wer_bpe:396] predicted :she scored nine for the queries


...

[NeMo I 2023-05-21 09:44:53 nemo_model_checkpoint:168] New best .nemo model saved to: /home/experiments/nvidia-stt-ft-02-spe/2023-05-21_09-33-30/checkpoints/nvidia-stt-ft-02-spe.nemo
[NeMo I 2023-05-21 09:44:56 nemo_model_checkpoint:168] New best .nemo model saved to: /home/experiments/nvidia-stt-ft-02-spe/2023-05-21_09-33-30/checkpoints/nvidia-stt-ft-02-spe.nemo


`Trainer.fit` stopped: `max_epochs=10` reached.


[NeMo I 2023-05-21 09:45:00 nemo_model_checkpoint:168] New best .nemo model saved to: /home/experiments/nvidia-stt-ft-02-spe/2023-05-21_09-33-30/checkpoints/nvidia-stt-ft-02-spe.nemo


Restoring states from the checkpoint path at /home/experiments/nvidia-stt-ft-02-spe/2023-05-21_09-33-30/checkpoints/nvidia-stt-ft-02-spe--val_wer=0.0049-epoch=8.ckpt
Restored all states from the checkpoint file at /home/experiments/nvidia-stt-ft-02-spe/2023-05-21_09-33-30/checkpoints/nvidia-stt-ft-02-spe--val_wer=0.0049-epoch=8.ckpt


CPU times: user 8min 26s, sys: 49.4 s, total: 9min 15s
Wall time: 11min 29s


### Model Evaluation

In [37]:
# Evaluate model on the validation set
trainer.test(asr_model)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[NeMo I 2023-05-21 09:47:12 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-21 09:47:12 rnnt_wer_bpe:395] reference :
[NeMo I 2023-05-21 09:47:12 rnnt_wer_bpe:396] predicted :he wrote a book with nine chapters
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:395] reference :
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:396] predicted :she spent nine days this month volunteering at an animal shelter
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:395] reference :
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:396] predicted :he had to solve three math problems to pass the test
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:395] reference :
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:396] predicted :he held up three fingers to indicate how many cupcakes he wanted
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:394] 
    
[NeMo I 2023-05-21 09:47:13 rnnt_wer_bpe:395] reference :
[NeM

[NeMo W 2023-05-21 09:49:28 nemo_logging:349] /opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
      rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
    


In [69]:
import numpy as np
import noisereduce as nr
import soundfile as sf
from pyloudnorm import Meter

# Convert manifest files to json format
def manifest_tojson(manifest_file: str) -> str:
    manifest_list = []
    with open(manifest_file, "r") as file:
        for line in file:
            record = json.loads(line)
            manifest_list.append(record)
    return manifest_list

def normalize_to_til(transcript: str) -> str:
    # TIL output is purely uppercase alphabet and space
    # so we normalize the output to that
    result = "".join([c.upper() if c.isalpha() else " " for c in transcript])
    # Remove double spaces
    while "  " in result:
        result = result.replace("  ", " ")
    return result

def asr_predict(manifest_file, start_idx, end_idx, batch_size=4, force_loudness=False, force_denoise=False):
    predicted_transcripts = []
    paths = []
    count = 0
    
    batched_records = [manifest_file[i:i+batch_size] for i in range(start_idx, end_idx, batch_size)]
    
    for batch_records in batched_records:
        batch_paths = []
        for record in batch_records:
            path = record['audio_filepath']
            if force_denoise:
                data, rate = sf.read(path)
                data = nr.reduce_noise(audio_clip=data, noise_clip=data, verbose=False)
                path = path.replace(".wav", "_denoised.wav")
                sf.write(path, data, rate)
            batch_paths.append(path)

        transcripts, metadata = asr_model.transcribe(batch_paths)

        for transcript in transcripts:
            pred = normalize_to_til(transcript)
            if pred == "" or force_loudness:
                count +=1
                data, rate = sf.read(path)
                meter = Meter(rate)
                loudness = meter.integrated_loudness(data)
                print("Loudness:", loudness)

                best_transcription = ""
                best_target_loudness = None

                for target_loudness in (-24, -14, -11, -9, -8, -7):
                    print("Trying target:", target_loudness)
                    gain = target_loudness - loudness
                    normalized_data = data * 10**(gain/20.0)
                    path = path.replace(".wav", "_normalized.wav")
                    sf.write(path, normalized_data, rate)
                    transcript, metadata = asr_model.transcribe([path])
                    pred = normalize_to_til(transcript[0])
                    if len(pred) > len(best_transcription):
                        best_transcription = pred
                        best_target_loudness = target_loudness

                print("After normalisation:", best_transcription)
                print("Best target loudness:", best_target_loudness)
                predicted_transcripts.append(best_transcription)
            else:
                predicted_transcripts.append(pred)
            paths.append(record['audio_filepath'][16:])
    print("Number of audio files too soft:", count)
    return predicted_transcripts, paths

TEST_MANIFEST = manifest_tojson(test_manifest_file)

In [70]:
import math
BATCH_SIZE = 1000
start_idx = 0
end_idx = BATCH_SIZE
iterations = math.ceil(len(TEST_MANIFEST) / BATCH_SIZE)
all_transcripts = []
all_paths = []
for batch in range(iterations):
    print(f'Batch {batch+1}: Records {start_idx} to {end_idx}')
    predicted_transcripts, paths = asr_predict(TEST_MANIFEST, start_idx, end_idx)
    all_transcripts.extend(predicted_transcripts)
    all_paths.extend(paths)
    start_idx += BATCH_SIZE
    end_idx += BATCH_SIZE

Batch 1: Records 0 to 1000
Loudness: -47.22134822543276
Trying target: -24
After normalisation: SHE HAD TO TAKE THREE DIFFERENT MEDICATIONS TO STAY HEALTHY
Loudness: -38.100571150829346
Trying target: -24
Trying target: -14
Trying target: -11
Trying target: -9
Trying target: -8
Trying target: -7
After normalisation: THEY DECIDED TO ADOPT TWO KITTENS FROM THE ANIMAL SHELTER
Number of audio files too soft: 2
Batch 2: Records 1000 to 2000
Loudness: -52.841079251785516
Trying target: -24
After normalisation: THEY HAD TO WAIT EIGHT DAYS FOR THEIR PASSPORT TO ARRIVE
Loudness: -47.079260251529874
Trying target: -24
After normalisation: HE HAS EIGHT SIBLINGS AND HE IS THE ELDEST
Loudness: -46.79070506877805
Trying target: -24
After normalisation: HAD EIGHT DIFFERENT OUTFITS FOR THE PHOTOSHOOT
Loudness: -35.539001506500895
Trying target: -24
After normalisation: HAD TO READ SEVEN BOOKS BEFORE HER ENGLISH CLASS
Loudness: -42.166105297314054
Trying target: -24
After normalisation: HE HAD TO TAKE 

In [ ]:
# Combine all lists of transcripts into a single list to update the annotation column
test_set['annotation'] = list(np.concatenate(all_transcripts).flat)
# Output the submission csv file
test_set.to_csv('eval_submission_nvidia_stt.csv', index=False)

In [72]:
import csv
def create_csv(paths, predictions, output_filename):
    # Ensure that paths and predictions are of the same length
    assert len(paths) == len(predictions), "Lists must have the same length"

    with open(output_filename, 'w', newline='') as csvfile:
        fieldnames = ['path', 'annotation']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for path, prediction in zip(paths, predictions):
            writer.writerow({'path': path, 'annotation': prediction})
create_csv(all_paths, all_transcripts, 'eval_submission_nvidia_conformer_transducer_ft_02_normalised.csv')